# Market data download

In [10]:
import pandas as pd
ticker_data = pd.read_csv('data/nasdaq_screener_1717056609861.csv')
top_cap = ticker_data.sort_values('Market Cap', ascending=False).iloc[:200]
top_tickers = top_cap['Symbol']

In [11]:
import pandas as pd
import yfinance as yf
import numpy as np

start_date = '2022-08-01'
end_date = '2023-12-31'

# Replace with your chosen stock tickers
stock_tickers = top_tickers.values

def get_stock_data(stock_ticker, start_date, end_date):
    stock_data = yf.download(stock_ticker, start=start_date, end=end_date)
    return stock_data['Close']

def get_risk_free_rate(ticker="^TNX", start_date="2023-01-01", end_date="2023-12-31"):
    risk_free_data = yf.download(ticker, start=start_date, end=end_date)
    return risk_free_data['Close']

def calculate_historical_volatility(stock_prices, window=63):
    daily_returns = stock_prices.pct_change().dropna()
    historical_volatility = daily_returns.rolling(window=window).std() * np.sqrt(252)
    return historical_volatility.dropna()

data = []

for ticker in stock_tickers:
    stock_prices = get_stock_data(ticker, start_date, end_date)
    risk_free_rates = get_risk_free_rate(start_date=start_date, end_date=end_date)
    historical_volatility = calculate_historical_volatility(stock_prices)

    dates = historical_volatility.index
    prices_at_dates = stock_prices.loc[dates]
    rates_at_dates = risk_free_rates[dates].fillna(method='ffill')

    for date, price, rate, volatility in zip(dates, prices_at_dates, rates_at_dates, historical_volatility):
        data.append([date, ticker, rate, price, volatility])

df = pd.DataFrame(data, columns=['date', 'ticker', 'risk_free_rate', 'spot_price', 'volatility'])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [12]:
df.to_csv('data/spot_price.csv', index=False)

# Option contract simulation

In [13]:
import numpy as np
import pandas as pd

from datetime import datetime
from dateutil.relativedelta import relativedelta

df = pd.read_csv('data/spot_price.csv')

date_format = "%Y-%m-%d"
df['date'] = df['date'].apply(lambda x: datetime.strptime(x, date_format))

strikes = pd.DataFrame([1.0], columns=['strike'])
maturities = pd.DataFrame([relativedelta(days=2), relativedelta(weeks=2), relativedelta(months=1)], columns=['maturity'])

option_df_tmp1 = df.merge(maturities, 'cross')
option_df_tmp1 ['expiration_date'] = option_df_tmp1['date'] + option_df_tmp1['maturity']
option_df_tmp1['expiration_date'] = option_df_tmp1['expiration_date'].apply(lambda x: datetime.strptime(str(x)[:-9], date_format))

sample_a = option_df_tmp1
sample_b = df[['date','ticker', 'spot_price']].rename({'date':'expiration_date'}, axis=1)
option_df_tmp2 = pd.merge(sample_a, sample_b, on=['expiration_date', 'ticker'], how='inner', suffixes=['','_expiration'])

option_df_tmp3 = option_df_tmp2.merge(strikes, 'cross')
option_df_tmp3.head()

c:\Users\vslys\anaconda3\lib\site-packages\pandas\core\arrays\datetimelike.py:1189: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  warnings.warn(


,date,ticker,risk_free_rate,spot_price,volatility,maturity,expiration_date,spot_price_expiration,strike
0,2022-10-28,MSFT,4.010,235.869995,0.348408,relativedelta(days=+14),2022-11-11,247.110001,1.0
1,2022-11-09,MSFT,4.151,224.509995,0.363603,relativedelta(days=+2),2022-11-11,247.110001,1.0
2,2022-10-28,MSFT,4.010,235.869995,0.348408,relativedelta(months=+1),2022-11-28,241.759995,1.0
3,2022-11-14,MSFT,3.865,241.550003,0.403852,relativedelta(days=+14),2022-11-28,241.759995,1.0
4,2022-10-31,MSFT,4.077,232.130005,0.348984,relativedelta(days=+2),2022-11-02,220.100006,1.0


In [14]:
option_df_tmp3['strike'] = option_df_tmp3['strike'] + np.random.normal(loc=0, scale=0.05,size=option_df_tmp3.shape[0])
option_df_tmp3['option_payout'] = (option_df_tmp3['spot_price_expiration'] - option_df_tmp3['strike'] * option_df_tmp3['spot_price']).clip(0.0)
option_df_tmp3['time_to_maturity'] = (option_df_tmp3['expiration_date'] - option_df_tmp3['date']).apply(lambda x: x.days)/252
option_df_tmp3['strike'] = option_df_tmp3['strike'] * option_df_tmp3['spot_price']
option_df = option_df_tmp3.drop(['expiration_date', 'spot_price_expiration', 'maturity'], axis=1)
option_df.head(5)

,date,ticker,risk_free_rate,spot_price,volatility,strike,option_payout,time_to_maturity
0,2022-10-28,MSFT,4.010,235.869995,0.348408,230.492726,16.617275,0.055556
1,2022-11-09,MSFT,4.151,224.509995,0.363603,224.718940,22.391060,0.007937
2,2022-10-28,MSFT,4.010,235.869995,0.348408,237.396991,4.363003,0.123016
3,2022-11-14,MSFT,3.865,241.550003,0.403852,268.822980,0.000000,0.055556
4,2022-10-31,MSFT,4.077,232.130005,0.348984,244.299557,0.000000,0.007937


In [15]:
option_df.shape

(119982, 8)

In [16]:
option_df.to_csv('data/option_features.csv', index=False)